In [1]:
# Importing the Libraries
import warnings
import logging
import pyspark
from pyspark.sql import SparkSession
from datetime import datetime
import config  # Contains user and password for MySQL

warnings.filterwarnings("ignore")  # Suppress specific warnings
logging.basicConfig(level=logging.WARN)  # Set the logging level

# Configure logging
logging.basicConfig(filename='customer_module.log', format='%(asctime)s - %(message)s', level=logging.INFO)


In [ ]:
def get_customer_details():
    while True:
        first_name = input("Enter First Name: ").strip()
        if first_name.isalpha():  # Check if all characters are alphabetic
            break
        else:
            print("Invalid input. First name should contain only alphabetic characters.")
    
    while True:
        last_name = input("Enter Last Name: ").strip()
        if last_name.isalpha():  # Check if all characters are alphabetic
            break
        else:
            print("Invalid input. Last name should contain only alphabetic characters.")
    
    while True:
        zip_code = input("Enter ZIP Code (for example: 01824): ").strip()
        if zip_code.isdigit() and len(zip_code) == 5:  # Assuming ZIP code is 5 digits
            break
        else:
            print("Invalid ZIP Code. Please enter a 5-digit numeric ZIP Code.")
    
    while True:
        last_4_cc = input("Enter Last 4 Digits of Credit Card Number: ").strip()
        if last_4_cc.isdigit() and len(last_4_cc) == 4:  # Assuming last 4 digits of CC are exactly 4 digits
            break
        else:
            print("Invalid input. Please enter exactly 4 digits for the credit card number.")
    
    return first_name, last_name, zip_code, last_4_cc


In [ ]:
# Function to check if entered user details matches a record in the database
def is_customer_valid(first_name, last_name, zip_code, last_4_cc, spark):
    jdbc_url = "jdbc:mysql://localhost:3306/creditcard_capstone"
    query = f"SELECT 1 FROM cdw_sapp_customer cust " \
            f"JOIN cdw_sapp_credit_card cc ON cust.CREDIT_CARD_NO = cc.CUST_CC_NO " \
            f"WHERE cust.FIRST_NAME = '{first_name}' AND cust.LAST_NAME = '{last_name}' " \
            f"AND cust.CUST_ZIP = '{zip_code}' AND SUBSTR(cc.CUST_CC_NO, -4) = '{last_4_cc}' LIMIT 1"

    user = config.user  # Ensure config is imported or defined correctly
    password = config.password  # Ensure config is imported or defined correctly

    try:
        df = spark.read.format("jdbc") \
            .option("url", jdbc_url) \
            .option("query", query) \
            .option("user", user) \
            .option("password", password) \
            .load()

        return not df.isEmpty()

    except Exception as e:
        error_msg = f"Error: Customer not found in database: {str(e)}"
        print(error_msg)
        logging.error("%s at %s.", error_msg, datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
        return False

In [ ]:
# Function to modify existing account details of a customer
def modify_account_details(first_name, last_name, zip_code, last_4_cc, new_data, spark):
    # JDBC parameters
    jdbc_url = "jdbc:mysql://localhost:3306/creditcard_capstone"

    table_name = "cdw_sapp_customer"

    user = config.user
    password = config.password

    try:
        # Creating a temporary view of the DataFrame to execute SQL queries
        new_data.createOrReplaceTempView("new_data_view")

        # Execute update query to modify account details
        spark.sql(f"UPDATE {table_name} SET FIRST_NAME = (SELECT FIRST_NAME FROM new_data_view), LAST_NAME = (SELECT LAST_NAME FROM new_data_view) WHERE FIRST_NAME = '{first_name}' AND LAST_NAME = '{last_name}' AND CUST_ZIP = '{zip_code}' AND SUBSTR(CUST_CC_NO, -4) = '{last_4_cc}'")

        # Logging successful modification of account details with timestamp
        logging.info("Account details modified successfully at %s.", datetime.now().strftime('%Y-%m-%d %H:%M:%S'))

        return True

    except Exception as e:
        print(f"Error modifying account details: {str(e)}")
        # Logging error while modifying account details with timestamp
        logging.error(f"Error modifying account details: {str(e)} at %s.", datetime.now().strftime('%Y-%m-%d %H:%M:%S'))


In [3]:
# Function to check existing account details of a customer
def check_account_details(first_name, last_name, zip_code, last_4_cc, spark):
    # JDBC parameters
    jdbc_url = "jdbc:mysql://localhost:3306/creditcard_capstone"

    dbtable_query = f"(SELECT * FROM cdw_sapp_customer cust JOIN cdw_sapp_credit_card cc ON cust.CREDIT_CARD_NO = cc.CUST_CC_NO WHERE cust.FIRST_NAME = '{first_name}' AND cust.LAST_NAME = '{last_name}' AND cust.CUST_ZIP = '{zip_code}' AND SUBSTR(cc.CUST_CC_NO, -4) = '{last_4_cc}') AS account_details"

    user = config.user
    password = config.password

    try:
        df = spark.read.format("jdbc") \
            .option("url", jdbc_url) \
            .option("dbtable", dbtable_query) \
            .option("user", user) \
            .option("password", password) \
            .load()

        # Logging successful query of account details with timestamp
        logging.info("Account details fetched successfully at %s.", datetime.now().strftime('%Y-%m-%d %H:%M:%S'))

        return df

    except Exception as e:
        print(f"Error querying account details: {str(e)}")
        # Logging error while querying account details with timestamp
        logging.error(f"Error querying account details: {str(e)} at %s.", datetime.now().strftime('%Y-%m-%d %H:%M:%S'))

        return None

In [5]:
# Function to generate monthly bill for a customer for a given month and year
def generate_monthly_bill(first_name, last_name, zip_code, last_4_cc, month, year, spark):
    # JDBC parameters
    jdbc_url = "jdbc:mysql://localhost:3306/creditcard_capstone"

    dbtable_query = f"(SELECT * FROM cdw_sapp_credit_card cc JOIN cdw_sapp_customer cust ON cust.CREDIT_CARD_NO = cc.CUST_CC_NO WHERE cust.FIRST_NAME = '{first_name}' AND cust.LAST_NAME = '{last_name}' AND cust.CUST_ZIP = '{zip_code}' AND SUBSTR(cc.CUST_CC_NO, -4) = '{last_4_cc}' AND cc.month = {month} AND cc.year = {year}) AS monthly_bill"

    user = config.user
    password = config.password

    try:
        df = spark.read.format("jdbc") \
            .option("url", jdbc_url) \
            .option("dbtable", dbtable_query) \
            .option("user", user) \
            .option("password", password) \
            .load()

        # Logging successful generation of monthly bill with timestamp
        logging.info("Monthly bill generated successfully at %s.", datetime.now().strftime('%Y-%m-%d %H:%M:%S'))

        return df

    except Exception as e:
        print(f"Error generating monthly bill: {str(e)}")
        # Logging error while generating monthly bill with timestamp
        logging.error(f"Error generating monthly bill: {str(e)} at %s.", datetime.now().strftime('%Y-%m-%d %H:%M:%S'))

        return None

In [6]:
# Function to display transactions made by a customer between two dates
def display_transactions(first_name, last_name, zip_code, last_4_cc, start_date, end_date, spark):
    # JDBC parameters
    jdbc_url = "jdbc:mysql://localhost:3306/creditcard_capstone"

    dbtable_query = f"(SELECT * FROM cdw_sapp_credit_card cc JOIN cdw_sapp_customer cust ON cust.CREDIT_CARD_NO = cc.CUST_CC_NO WHERE cust.FIRST_NAME = '{first_name}' AND cust.LAST_NAME = '{last_name}' AND cust.CUST_ZIP = '{zip_code}' AND SUBSTR(cc.CUST_CC_NO, -4) = '{last_4_cc}' AND cc.DAY BETWEEN {start_date} AND {end_date} ORDER BY cc.YEAR DESC, cc.MONTH DESC, cc.DAY DESC) AS customer_transactions"

    user = config.user
    password = config.password

    try:
        df = spark.read.format("jdbc") \
            .option("url", jdbc_url) \
            .option("dbtable", dbtable_query) \
            .option("user", user) \
            .option("password", password) \
            .load()

        # Logging successful display of transactions with timestamp
        logging.info("Transactions displayed successfully at %s.", datetime.now().strftime('%Y-%m-%d %H:%M:%S'))

        return df

    except Exception as e:
        print(f"Error displaying transactions: {str(e)}")
        # Logging error while displaying transactions with timestamp
        logging.error(f"Error displaying transactions: {str(e)} at %s.", datetime.now().strftime('%Y-%m-%d %H:%M:%S'))

        return None

In [7]:
# Master function to orchestrate customer operations
def customer_operations(spark):
    while True:
        print("\nCustomer Operations Menu:")
        print("1. Check Account Details")
        print("2. Modify Account Details")
        print("3. Generate Monthly Bill")
        print("4. Display Transactions Between Two Dates")
        print("5. Exit")

        choice = input("Enter your choice (1-5): ").strip()

        if choice == '1':
            first_name, last_name, zip_code, last_4_cc = get_customer_details()
            df = check_account_details(first_name, last_name, zip_code, last_4_cc, spark)
            if df is not None:
                df.show(truncate=False)
        elif choice == '2':
            first_name, last_name, zip_code, last_4_cc = get_customer_details()
            new_first_name = input("Enter New First Name: ").strip()
            new_last_name = input("Enter New Last Name: ").strip()
            new_data = spark.createDataFrame([(new_first_name, new_last_name)], ["FIRST_NAME", "LAST_NAME"])
            if modify_account_details(first_name, last_name, zip_code, last_4_cc, new_data, spark):
                print("Account details modified successfully.")
        elif choice == '3':
            first_name, last_name, zip_code, last_4_cc = get_customer_details()
            month = int(input("Enter Month (1-12): ").strip())
            year = int(input("Enter Year: ").strip())
            df = generate_monthly_bill(first_name, last_name, zip_code, last_4_cc, month, year, spark)
            if df is not None:
                df.show(truncate=False)
        elif choice == '4':
            first_name, last_name, zip_code, last_4_cc = get_customer_details()
            start_date = input("Enter Start Date (YYYY-MM-DD): ").strip()
            end_date = input("Enter End Date (YYYY-MM-DD): ").strip()
            df = display_transactions(first_name, last_name, zip_code, last_4_cc, start_date, end_date, spark)
            if df is not None:
                df.show(truncate=False)
        elif choice == '5':
            break
        else:
            print("Invalid choice. Please enter a number from 1 to 5.")

In [9]:
# Main
if __name__ == "__main__":
    # Creating Spark Session
    spark = SparkSession.builder \
        .appName('Customer Operations') \
        .getOrCreate()

    try:
        customer_operations(spark)
    finally:
        # Stop Spark session
        spark.stop()


Customer Operations Menu:
1. Check Account Details
2. Modify Account Details
3. Generate Monthly Bill
4. Display Transactions Between Two Dates
5. Exit
+--------------+---+-----+----+--------+----------------+---------+-----------+----------------+-----------------+----------+-----------+---------+---------+----------------+------+-----------------+---------+----------+-------------+--------+-------------+-------------------+-------------------+----------------------+
|TRANSACTION_ID|DAY|MONTH|YEAR|TIMEID  |CUST_CC_NO      |CUST_SSN |BRANCH_CODE|TRANSACTION_TYPE|TRANSACTION_VALUE|FIRST_NAME|MIDDLE_NAME|LAST_NAME|SSN      |CREDIT_CARD_NO  |APT_NO|STREET_NAME      |CUST_CITY|CUST_STATE|CUST_COUNTRY |CUST_ZIP|CUST_PHONE   |CUST_EMAIL         |LAST_UPDATED       |FULL_STREET_ADDRESS   |
+--------------+---+-----+----+--------+----------------+---------+-----------+----------------+-----------------+----------+-----------+---------+---------+----------------+------+-----------------+------